In [78]:
from tba_pydata import api, analysis
from tba_pydata.games import power_up_2018 as powerup

import pandas
from numpy import tanh, arctanh
from scipy.stats import t

In [17]:
PICKLE_PATH = '2018wk1.pickle'
#df = api.matches(year=2018, event_predicate=(lambda e: e.week==0.0), score_parsing_fn=powerup.scoring
df = pandas.read_pickle(PICKLE_PATH)

In [41]:
mdl = analysis.get_epr_model(df, score_col='total_points')

/home/wes/Development/frc/TBA-PyData/tba_pydata/analysis.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[margin] = df[red_score] - df[blue_score]


In [42]:
mdl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     41.09
Date:                Tue, 06 Mar 2018   Prob (F-statistic):               0.00
Time:                        11:24:22   Log-Likelihood:                -27842.
No. Observations:                4872   AIC:                         5.750e+04
Df Residuals:                    3964   BIC:                         6.339e+04
Df Model:                         908                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
frc1          72.5877     17.805      4.077      0.000      37.680     107.495
frc1002       44.8631     18.070      2.483      0.013       9.436      80.290
frc1014      177.0300     22.659      7.813      0.000     132.607     221.453
frc1023      177.4804     17.613     10.076      0.000     142.948     212.012
frc1038      129.4654     22.421      5.774      0.000      85.508     173.423
frc1051      137.2708     23.050      5.955      0.000      92.080     182.462
frc1058      173.3420     17.696      9.795      0.000     138.647     208.037
frc1073       97.8622     17.516      5.587      0.000      63.522     132.203
frc11         51.5243     21.591      2.386      0.017       9.193      93.856
frc1100      104.3043     17.607      5.924      0.000      69.786     138.823
frc1102      133.6729     17.392      7.686      0.000      99.574     167.772
frc1137        7.0335     17.647      0.399      0.690     -27.564      41.631
frc115       132.0572     21.979      6.008      0.000      88.966     175.148
frc1153       89.8568     17.884      5.024      0.000      54.794     124.920
frc116        45.7034     18.131      2.521      0.012      10.156      81.250
frc118       214.7848     22.574      9.515      0.000     170.527     259.042
frc1188       90.0440     17.294      5.207      0.000      56.138     123.950
frc122        93.3183     17.754      5.256      0.000      58.511     128.126
frc1225      100.0007     17.753      5.633      0.000      65.196     134.806
frc123        74.2135     17.726      4.187      0.000      39.461     108.966
frc1246       82.5676     17.478      4.724      0.000      48.302     116.833
frc1247       50.8610     17.845      2.850      0.004      15.875      85.846
frc125       188.3905     22.789      8.267      0.000     143.712     233.069
frc1255       31.1569     22.510      1.384      0.166     -12.976      75.290
frc126        34.4053     17.511      1.965      0.050       0.074      68.736
frc1277       10.7090     17.522      0.611      0.541     -23.643      45.061
frc128         6.4661     22.737      0.284      0.776     -38.112      51.044
frc1285      130.0252     17.550      7.409      0.000      95.617     164.433
frc1287      115.4031     22.773      5.067      0.000      70.754     160.052
frc1289       62.2874     17.345      3.591      0.000      28.281      96.294
frc1294       22.1726     17.501      1.267      0.205     -12.140      56.485
frc1296      109.1358     22.260      4.903      0.000      65.493     152.779
frc1305      208.4125     17.559     11.869      0.000     173.988     242.837
frc1307       38.1577     17.487      2.182      0.029       3.873      72.442
frc131        82.9586     17.551      4.727      0.000      48.548     117.369
frc1310      147.1540     17.986      8.182      0.000     111.892     182.416
fr

In [48]:
len(mdl.params)

908

In [59]:
def epr_make_predictions(red, blue, model):
    table = [dict.fromkeys(model.params.index, 0) for i in range(3)]
    for r, b in zip(red, blue):
        table[0][r] = 1
        table[1][r] = 1
        table[1][b] = -1
        table[2][b] = 1
    x = pandas.DataFrame.from_dict(table)
    return model.get_prediction(x)

In [ ]:
df.to_pickle(PICKLE_PATH)

In [96]:
pred = epr_make_predictions(['frc422', 'frc3274', 'frc141'],['frc2363', 'frc1418'], mdl)
predt = pred.predicted_mean / (pred.var_pred_mean**0.5)
t.cdf(predt, df=10)

Index(['frc1', 'frc1002', 'frc1014', 'frc1023', 'frc1038', 'frc1051',
       'frc1058', 'frc1073', 'frc11', 'frc1100',
       ...
       'frc876', 'frc877', 'frc88', 'frc885', 'frc888', 'frc894', 'frc900',
       'frc94', 'frc95', 'frc968'],
      dtype='object', length=908)


array([ 0.99999688,  0.00165636,  0.99999998])

In [54]:
red = ['frc2363', 'frc1418', 'frc122']
blue = ['frc422', 'frc3274', 'frc141']
table = [dict.fromkeys(mdl.params.index, 0) for i in range(3)]
for r, b in zip(red, blue):
    table[0][r] = 1
    table[1][r] = 1
    table[1][b] = -1
    table[2][b] = 1
x = pandas.DataFrame.from_dict(table)
x.shape

(3, 908)

In [81]:
t.cdf(-2, df=9)

0.038276411885350463

In [85]:
mdl.df_model

908.0